In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.01

Capacitance = 2.
Inductance = 2.
R = 1

# solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "gp_canonical_X"
# solver = "qcqp_sni"
# solver = "bisection_b"
time_limit = 200

# Ciropt problem

In [3]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d
# bounds = {  
#             # 'b': {"ub": 10, "lb": 1},
#             # 'd': {"ub": 10, "lb": 0.},
#             'h': {"ub": 1, "lb": 1},
#             'alpha': {"ub": 0.5, "lb": 0.5},
#             'beta': {"ub": 0.5, "lb": 0.5},
#             }
# res, model, sp_exp = problem.solve(solver=solver, verbose=False, bounds=bounds, debug=True)
res, model, sp_exp = problem.solve(solver=solver, verbose=False, debug=True)

# res = params = {  'b': 10,
#             'h': 1,
#             'd': 0.,
#             'alpha': 0.5,
#             'beta': 0.5 }
# b = problem.solve(solver=solver, freq=1, max_iters=100, params=params, debug=True)
# print(f"{b = }")
# res["b"] = b

dim_G=6, dim_F=4


In [4]:
# list(problem.name2idx.keys())

In [5]:
# print(problem.opti.stats()["iter_count"])

# if solver == "ca":
#     P = model.value(vars['P_full'])
#     lamb = model.value(vars['lamb0'])
#     print(P.min(), P.max(), lamb.min(), lamb.max())

In [6]:
# 0.1
# {'b': 2.7852966369101124,
#  'h': 5.895098847742334,
#  'd': 0.8661256508753064,
#  'alpha': 0.5089330928372713,
#  'beta': 0.42928788709774646}

# 0.01
# {'b': 2.680175862785157,
#  'h': 5.925536207094371,
#  'd': 0.7873574606589531,
#  'alpha': 0.5047606321755108,
#  'beta': 0.45583144590263336}

# 0.0001
# {'b': 2.66846146628235,
#  'h': 6.035347263881326,
#  'd': 0.7721222423737911,
#  'alpha': 0.4915885826616601,
#  'beta': 0.4437526051891429}

# 0
# {'b': 2.667815555306872,
#  'h': 6.025408082246163,
#  'd': 0.7699747144077753,
#  'alpha': 0.49133542402684943,
#  'beta': 0.44275984288712256}

# PEP verification

In [7]:
res

{'b': 2.680175862785157,
 'h': 5.925536207094371,
 'd': 0.7873574606589531,
 'alpha': 0.5047606321755108,
 'beta': 0.45583144590263336}

In [8]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-4.2714510311496305e-10
